# import modules

In [1]:
from datetime import datetime
import pickle
import gzip
import numpy as np
from scipy.linalg import eig,norm
from math import *
from qutip import parfor,qeye,sigmax,sigmay,sigmaz
from qutip import *
np.set_printoptions(precision=3)
from IPython.display import display
import seaborn as sns


%matplotlib inline

import pandas as pd
from os import listdir
from os.path import isfile, join

# Analyze  data

## import data

In [32]:
N = 5 # measurements repeats number
mypath = './data/test/'

theta = 'theta=1.1'
files = [f for f in listdir(mypath) if theta in f]

df = pd.DataFrame(columns=('measures', 'S0','Sx+','Sy+'))

BP = []
W = []
Rho = []

for fname in files[0:100]:
    f = gzip.open(mypath+fname, 'rb')
    head = pickle.load(f)
    for i in range(N):
        t, ii , data = pickle.load(f)
        m = np.average((data > 1.5)*1.0,axis=0)[1::2]
        df.loc[len(df)+1] = [i] + list(m)
    f.close()

    Z = df.mean(axis=0)[1]  
    X = df.mean(axis=0)[2]
    Y = df.mean(axis=0)[3]
    Pz = 1-2*Z
    Px = 2*X-1
    Py = 2*Y-1
    rho = (qeye(2)+Px*sigmax()+Py*sigmay()+Pz*sigmaz())/2
    Rho.append(rho)
    bp = np.angle(rho[1][0][0])-0*pi
    w = abs(rho[1][0][0])/0.5
    BP.append(bp)
    W.append(w)
data = {'Berry Phase':BP, 'W':W}
Data = pd.DataFrame(data)
# df.to_csv('measure_bp.csv')

In [33]:
rho_aver = 0
P = len(Rho)
for j in Rho[:P]:
    rho_aver += j
rho_aver = rho_aver / P
print(np.angle(rho_aver[1][0][0]) - 0*pi)
#     print(-4*pi*cos(Theta))
print(abs(rho_aver[1][0][0])/0.5)

1.01750200147
0.959099577729


In [ ]:
(0.972,0.977,0.969,0.973),(0.965,0.968,0.971,0.971),(0.938,0.929,0.942,0.933),(0.898,0.901,0.894,0.891),(0.832,0.833,0.828)

## Tomography

## 非极大似然估计

## 极大似然估计

In [30]:
# 2017-11-10 修正正定问题
import qutip as qu
import qinfer as qi 


def ML_EstimateState(n,N,MeasurBasis):
    ''' Give result and measurement Basis to estimate the density matrix of the state
    
    n: success measurement shots
    N: total measurement shots
    MeasurBasis: measurement basis used in the experiment
    '''
    # judge how many qubits
    dims = MeasurBasis[0].dims[0]
    dim = np.product(dims)
    
    def Gen_GM_Basis (dims): 
        ibasis = qi.tomography.gell_mann_basis(dims[0])
        ibasis = ibasis[range(len(ibasis))]
        if (len(dims) > 1):
            return [ qu.tensor(e, e1) for e in ibasis for e1 in  Gen_GM_Basis(dims[1:])]
        else:
            return ibasis
        
    B_all = Gen_GM_Basis(dims)
    B0 = B_all[0]   #  1/d matrix, d is the dimension = 2**N
    B  = B_all[1:]  # pauli matrix list, length d-1
    # generate tranform matrix X
    X  = [[(qu.ket2dm(Pi)*Bj).tr() for Bj in B] for Pi in MeasurBasis]
    X  = np.array(X)
    
    f = (n+0.5)/(N+1)
    Y = f- 1.0/dim
    
    a = np.sqrt(N/(f*(1-f)))
    Y1 = a*Y
    X1 = np.array([np.array(Xk)*ak for Xk,ak in zip(X,a)])
    
    # calculate initial value by linear transform
    #x00 = np.real((np.linalg.inv((X.T.dot(X))).dot(X.T)).dot(Y))
    x00 = np.zeros(dim**2-1)
    
    from scipy.optimize import minimize
    # estimate bound
    #bound = [(0.0, None) for x in x00]

    # estimate constraints
    def con_fun(x):
        d = np.sqrt(len(x)+1)
        return 1.0-1.0/d - np.linalg.norm(x, ord=2)**2
    constrain = {
        'type': 'ineq',
        'fun': con_fun
    }
    # estimate function
    def estimate_fun(theta):
        return (np.linalg.norm(Y1-X1.dot(theta),ord=2))
    # to estimate values
#     res = minimize(estimate_fun,x00,constraints=constrain, method = 'SLSQP')
    res = minimize(estimate_fun,x00,constraints=constrain, method = 'COBYLA')
    rho = np.sum([x*Bi for x, Bi in zip(res.x,B)],axis=0) + B0*B0
    return (rho,res)

In [31]:
import numpy as np
from qutip import basis,fidelity
s0=basis(2,0)
s1=basis(2,1)
s2=basis(2,0)+basis(2,1)
s3=basis(2,0)+1.0j*basis(2,1)
s4=basis(2,0)-basis(2,1)
s5=basis(2,0)-1.0j*basis(2,1)

s2 /= s2.norm()
s3 /= s3.norm()
s4 /= s4.norm()
s5 /= s5.norm()


proj=[s0,s2,s3]
y = np.array([Z,X,Y])
N =10000*np.ones(3)

# proj=[s0,s1,s3,s2,s5,s4]
# y = np.array([0.45,  0.49,  0.97,  0.51,  0.00,  0.51]) # X state
# N =10000*np.ones(6)

# proj=[s0,s1,s3,s2]
# y = np.array([0.45,  0.49,  0.97,  0.51])
# N =10000*np.ones(4)

# proj=[s0,s1,s5,s4]
# y = np.array([0.45,  0.49,  0,  0.51])
# N =10000*np.ones(4)

a,b = ML_EstimateState(y*10000,N,proj)

print(df.iloc[0].values[3:-2])
print(b.x)
# print((a*a).tr())
a.eigenenergies()
b

[ 0.    0.49]
[ 0.037 -0.7   -0.016]


     fun: 0.30636206443576858
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 113
  status: 1
 success: True
       x: array([ 0.037, -0.7  , -0.016])

In [32]:
a

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.526+0.j    -0.495+0.012j]
 [-0.495-0.012j  0.474+0.j   ]]

# Analyze Multi Date

In [35]:
rhos = []

for i in range(20):
    N = 10 # measurements repeats number
    # 
    df = pd.DataFrame(columns=('measures', 'S0', 'Sz','Sx+','Sy+','Sx-','Sy-'))

    f = gzip.open('./data/experment'+str(i)+'.dat', 'rb')
    head = pickle.load(f)
    for i in range(N):
        t, ii , data = pickle.load(f)
        m = np.average((data > 1.5)*1.0,axis=0)[1::2]
        df.loc[len(df)+1] = [i] + list(m)
    f.close()
    
    Z = df.mean(axis=0)[1]  
    X = df.mean(axis=0)[3]
    Y = df.mean(axis=0)[4]
    Pz = 1-2*Z
    Px = 2*X-1
    Py = 2*Y-1
    rho = (qeye(2)+Px*sigmax()+Py*sigmay()+Pz*sigmaz())/2
    
    rhos.append(rho)

In [36]:
Rho = 0
P = len(rhos)
for i in range(P):
    Rho += rhos[i]
Rho /= P
Rho

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.626+0.j    -0.059+0.105j]
 [-0.059-0.105j  0.374+0.j   ]]